# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse   Shipments           Customer  \
0  Feb  2 2023  8:30AM  256430        10  0086346479  ISDIN Corporation   
1  Feb  2 2023  8:30AM  256430        10  0086346484  ISDIN Corporation   
2  Feb  2 2023  8:30AM  256430        10  0086346485  ISDIN Corporation   
3  Feb  2 2023  8:30AM  256430        10  0086346483  ISDIN Corporation   
4  Feb  2 2023  8:30AM  256440        10  0086346497  ISDIN Corporation   
5  Feb  2 2023  8:30AM  256440        10  0086346509  ISDIN Corporation   
6  Feb  2 2023  8:30AM  256440        10  0086346510  ISDIN Corporation   
7  Feb  2 2023  8:30AM  256440        10  0086346505  ISDIN Corporation   
8  Feb  2 2023  8:30AM  256440        10  0086346567  ISDIN Corporation   
9  Feb  2 2023  8:30AM  256440        10  0086346568  ISDIN Corporation   

  ShipmentStatus  
0       Released  
1       Released  
2       Released  
3       Released  
4       Released  
5       Released  
6       Released  
7       Released  
8       Released  
9       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
28  256434       Released          1
29  256436       Released          1
30  256437       Released          1
31  256438       Released          1
32  256440       Released          6

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
256434                NaN        NaN       1.0
256436                NaN        NaN       1.0
256437                NaN        NaN       1.0
256438                NaN        NaN       1.0
256440                NaN        NaN       6.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
256370                6.0        1.0      31.0
256391                0.0        1.0       0.0
256392                0.0        0.0       1.0
256393                9.0        2.0       2.0
256401                0.0        1.0       0.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
256370                  6          1        31
256391                  0          1         0
256392                  0          0         1
256393                  9          2         2
256401                  0          1         0

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               256370          6          1        31
1               256391          0          1         0
2               256392          0          0         1
3               256393          9          2         2
4               256401          0          1         0

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               256370         6         1       31
1               256391                   1         
2               256392                            1
3               256393         9         2        2
4               256401                   1

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse               Customer
0   Feb  2 2023  8:30AM  256430        10      ISDIN Corporation
4   Feb  2 2023  8:30AM  256440        10      ISDIN Corporation
12  Feb  1 2023  4:17PM  256438        16        SHL Pharma, LLC
13  Feb  1 2023  3:59PM  256437        19  ACG North America LLC
14  Feb  1 2023  3:58PM  256436        18     Innova Softgel LLC
15  Feb  1 2023  3:34PM  256434        22      NBTY Global, Inc.
16  Feb  1 2023  3:29PM  256433        22      NBTY Global, Inc.
17  Feb  1 2023  2:52PM  256427        19   Emersa Waterbox, LLC
18  Feb  1 2023  2:33PM  256420        19    AdvaGen Pharma, Ltd
19  Feb  1 2023  2:29PM  256419        19    AdvaGen Pharma, Ltd

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse               Customer Completed  \
0  Feb  2 2023  8:30AM  256430        10      ISDIN Corporation         2   
1  Feb  2 2023  8:30AM  256440        10      ISDIN Corporation             
2  Feb  1 2023  4:17PM  256438        16        SHL Pharma, LLC             
3  Feb  1 2023  3:59PM  256437        19  ACG North America LLC             
4  Feb  1 2023  3:58PM  256436        18     Innova Softgel LLC             
5  Feb  1 2023  3:34PM  256434        22      NBTY Global, Inc.             
6  Feb  1 2023  3:29PM  256433        22      NBTY Global, Inc.             
7  Feb  1 2023  2:52PM  256427        19   Emersa Waterbox, LLC             
8  Feb  1 2023  2:33PM  256420        19    AdvaGen Pharma, Ltd             
9  Feb  1 2023  2:29PM  256419        19    AdvaGen Pharma, Ltd             

  Executing Released  
0                  4  
1                  6  
2                  1  
3                  1  
4                  1  
5                  1  
6                  1  
7                  1  
8                  1  
9                  1

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse               Customer Released  \
0  Feb  2 2023  8:30AM  256430        10      ISDIN Corporation        4   
1  Feb  2 2023  8:30AM  256440        10      ISDIN Corporation        6   
2  Feb  1 2023  4:17PM  256438        16        SHL Pharma, LLC        1   
3  Feb  1 2023  3:59PM  256437        19  ACG North America LLC        1   
4  Feb  1 2023  3:58PM  256436        18     Innova Softgel LLC        1   
5  Feb  1 2023  3:34PM  256434        22      NBTY Global, Inc.        1   
6  Feb  1 2023  3:29PM  256433        22      NBTY Global, Inc.        1   
7  Feb  1 2023  2:52PM  256427        19   Emersa Waterbox, LLC        1   
8  Feb  1 2023  2:33PM  256420        19    AdvaGen Pharma, Ltd        1   
9  Feb  1 2023  2:29PM  256419        19    AdvaGen Pharma, Ltd        1   

  Executing Completed  
0                   2  
1                      
2                      
3                      
4                      
5                      
6                      
7                      
8                      
9

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse               Customer Released  \
0  Feb  2 2023  8:30AM  256430        10      ISDIN Corporation        4   
1  Feb  2 2023  8:30AM  256440        10      ISDIN Corporation        6   
2  Feb  1 2023  4:17PM  256438        16        SHL Pharma, LLC        1   
3  Feb  1 2023  3:59PM  256437        19  ACG North America LLC        1   
4  Feb  1 2023  3:58PM  256436        18     Innova Softgel LLC        1   

  Executing Completed  
0                   2  
1                      
2                      
3                      
4

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse               Customer  Released  \
0  Feb  2 2023  8:30AM  256430        10      ISDIN Corporation       4.0   
1  Feb  2 2023  8:30AM  256440        10      ISDIN Corporation       6.0   
2  Feb  1 2023  4:17PM  256438        16        SHL Pharma, LLC       1.0   
3  Feb  1 2023  3:59PM  256437        19  ACG North America LLC       1.0   
4  Feb  1 2023  3:58PM  256436        18     Innova Softgel LLC       1.0   

   Executing  Completed  
0        NaN        2.0  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse               Customer  Released  \
0  Feb  2 2023  8:30AM  256430        10      ISDIN Corporation       4.0   
1  Feb  2 2023  8:30AM  256440        10      ISDIN Corporation       6.0   
2  Feb  1 2023  4:17PM  256438        16        SHL Pharma, LLC       1.0   
3  Feb  1 2023  3:59PM  256437        19  ACG North America LLC       1.0   
4  Feb  1 2023  3:58PM  256436        18     Innova Softgel LLC       1.0   

   Executing  Completed  
0        0.0        2.0  
1        0.0        0.0  
2        0.0        0.0  
3        0.0        0.0  
4        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         1538457      53.0        5.0       22.0
12          512815       1.0        1.0        0.0
16          256438       1.0        0.0        0.0
18          256436       1.0        0.0        0.0
19         1794944      16.0        3.0       16.0
21          512783       1.0        1.0        0.0
22          512867       2.0        0.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  1538457      53.0        5.0       22.0
1        12   512815       1.0        1.0        0.0
2        16   256438       1.0        0.0        0.0
3        18   256436       1.0        0.0        0.0
4        19  1794944      16.0        3.0       16.0
5        21   512783       1.0        1.0        0.0
6        22   512867       2.0        0.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10      53.0        5.0       22.0
1        12       1.0        1.0        0.0
2        16       1.0        0.0        0.0
3        18       1.0        0.0        0.0
4        19      16.0        3.0       16.0
5        21       1.0        1.0        0.0
6        22       2.0        0.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   53.0
1        12  Released    1.0
2        16  Released    1.0
3        18  Released    1.0
4        19  Released   16.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10   12   16   18    19   21   22
Status                                        
Completed  22.0  0.0  0.0  0.0  16.0  0.0  0.0
Executing   5.0  1.0  0.0  0.0   3.0  1.0  0.0
Released   53.0  1.0  1.0  1.0  16.0  1.0  2.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10   12   16   18    19   21   22
0          Completed  22.0  0.0  0.0  0.0  16.0  0.0  0.0
1          Executing   5.0  1.0  0.0  0.0   3.0  1.0  0.0
2           Released  53.0  1.0  1.0  1.0  16.0  1.0  2.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10   12   16   18    19   21   22
0  Completed  22.0  0.0  0.0  0.0  16.0  0.0  0.0
1  Executing   5.0  1.0  0.0  0.0   3.0  1.0  0.0
2   Released  53.0  1.0  1.0  1.0  16.0  1.0  2.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()